In [1]:
import csv
import nltk
import pandas as pd

from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [2]:
# Load dataset
dataset = pd.read_csv('dataset_comentarios.csv', delimiter=';')

In [3]:
dataset

,ide_formulario_publicado,nom_titulo_formulario_publicado,des_conteudo,qtd_curtidas,qtd_descurtidas,dat_posicionamento,cod_autorizado
0,361682,PL 11098/2018,Não entendi muito bem a proposta.,0,0,2018-12-04 10:49:55,1
1,44036,PL 7180/2014,Acabar com doutrinação nas escolas,63,38,2018-12-04 10:51:56,1
2,44036,PL 7180/2014,Doutrinar crianças,17,35,2018-12-04 10:51:56,1
3,44036,PL 7180/2014,Professores têm que exercer a liberdade de Cát...,36,35,2018-12-04 10:55:44,1
4,360234,PL 10996/2018,privatiza de forma simplista o setor de saneam...,2,1,2018-12-04 11:05:08,1
...,...,...,...,...,...,...,...
381342,522936,PL 5411/2020,Vacinas experimentais. Sem consentimento ou an...,0,0,2021-07-08 11:01:16,0
381343,529652,INC 373/2021,Maior economia,0,0,2021-07-08 11:08:12,0
381344,395685,PL 533/2019,Enquanto universitário do curso de direito (UF...,0,0,2021-07-08 11:11:11,0
381345,529652,INC 373/2021,Muito bom pra carreira dos militares\n,0,0,2021-07-08 11:11:20,0


In [4]:
dataset_pl = dataset.loc[dataset['nom_titulo_formulario_publicado'] == 'PL 3723/2019']
sentences = list(dataset_pl['des_conteudo'])

In [5]:
sentences

['A Lei 10.826/2003 já define os atiradores desportivos, como categoria que tem direito ao porte federal, queremos nosso direito, isso não é um favor, queremos direito a legitima defesa.',
 'Atende parte dos anseios do povo',
 'Restaura o DIREITO do cidadão que cumpre os requisitos da Lei a portar armas de fogo para sua defesa.',
 'Da clareza ao texto original. Reconhece direitos fundamentais. Permite o acesso às armas pelo cidadão que cumpre a lei. Evita judicialização dos termos dúbios na norma atual.',
 'O estatuto do desarmamento em seu art. 6º, inciso IX, concede o porte de armas para os integrantes das entidades de desporto legalmente constituídas(CAC), faltando apenas a sua regulamentação. O PL 3723/19, oportunamente, vem sanar está \r\nomissão da administração.',
 'faltou a inclusão dos Auditores Fiscais dos Estados e Municípios, no inciso X, art. 6º, pois estão submetidos aos mesmos riscos que os Auditores Federais na realização de suas missões.',
 'Devolve ao cidadão comum, q

# Creating a vocabulary

In [6]:
# NLTK portuguese stopwords set
stop_words = set(stopwords.words('portuguese'))
stop_words.remove("não")

In [7]:
words = set()

for sentence in tqdm(sentences):
    if isinstance(sentence, str):
        cur_words = [word.lower() 
                     for word in 
                     nltk.word_tokenize(sentence) if 
                     (word.isalnum() and word.lower() not in stop_words)]
    words.update(cur_words)

100%|██████████| 705/705 [00:00<00:00, 4588.09it/s]


In [8]:
with open('vocabulary.txt', 'w') as vocabulary:
    for word in words:
        vocabulary.write(word)
        vocabulary.write('\n')

# Creating a .tsv

In [9]:
ren_sentences = []

for sentence in tqdm(sentences):
    if (isinstance(sentence, str)):
        cur_words = [word.lower() 
                     for word in 
                     nltk.word_tokenize(sentence) if 
                     (word.isalnum() and word.lower() not in stop_words)]
        
        new_sentence = ' '.join(cur_words)
        if (new_sentence.strip() != ""):
            ren_sentences.append(new_sentence)

100%|██████████| 705/705 [00:00<00:00, 7814.54it/s]


In [10]:
# Split train, test and validation
X_train, X_rem = train_test_split(ren_sentences, train_size=0.7)
X_test, X_val = train_test_split(X_rem, test_size=0.5)

In [11]:
# Add labels
train_list = [[sentence, 'train'] for sentence in X_train]
test_list = [[sentence, 'test'] for sentence in X_test]
val_list = [[sentence, 'val'] for sentence in X_val]

# Concat lists
concat_list = train_list + test_list + val_list

In [12]:
# Create Dataframe
corpus_df = pd.DataFrame(concat_list, columns=['text', 'train']).replace('"','', regex=True)
corpus_df.to_csv('corpus.tsv', sep='\t', index=False, header=False)
corpus_df

,text,train
0,restitui direito legítima defesa bandido não s...,train
1,segurança,train
2,2003 liberado todo cidadão assim devia voltar ...,train
3,morosidade votação,train
4,todos direito defender devem ter posse respons...,train
...,...,...
700,retorna cidadão bem direito proteção vida fami...,val
701,legitima atraves discussao legislativo estabel...,val
702,democracia permite direito escolha cidadão bem...,val
703,enho direito defender defender família não pol...,val
